In [9]:
%pip install -U --quiet langchain-google-community --user

Note: you may need to restart the kernel to use updated packages.


In [10]:
from dotenv import dotenv_values
import os

In [11]:
config = dotenv_values(".env")

In [31]:
os.environ["GOOGLE_API_KEY"] = config.get("GOOGLE_API_KEY")
os.environ["GOOGLE_CSE_ID"] = config.get("GOOGLE_CSE_ID")
openai_api_key = config["KEY_OPENAI"]

In [32]:
from langchain.retrievers.web_research import WebResearchRetriever
from langchain_chroma import Chroma
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_community.tools.google_search import GoogleSearchResults

In [33]:
# Vectorstore
vectorstore = Chroma(
    embedding_function=OpenAIEmbeddings(api_key=openai_api_key), persist_directory="./chroma_db_oai"
)

In [34]:
# LLM
llm = ChatOpenAI(temperature=0, api_key= openai_api_key, model="gpt-4")

# Search
search = GoogleSearchAPIWrapper()

In [41]:
# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore, llm=llm, search=search
)

In [36]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)
from langchain.chains import RetrievalQAWithSourcesChain

In [37]:
user_input = "What is the Síndrome SAPHO and what are the symptoms? Based on your findings write a description, main symptoms and how to treat them "
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever
)
result = qa_chain.invoke({"question": user_input})

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.17it/s]


In [38]:
result

{'question': 'What is the Síndrome SAPHO and what are the symptoms? Based on your findings write a description, main symptoms and how to treat them ',
 'answer': 'SAPHO syndrome is a complex disorder characterized by peculiar combinations of bone lesions and dermatologic manifestations. Patients typically present with musculo-skeletal complaints, usually of the anterior chest wall, synovial articulations and synchondroses. The disease is more common in the sterno-clavicular region, but can also affect the spine, pelvis and long bones. Various articular manifestations, often due to osteitis extending to adjacent articular structures, especially in the sacroiliac and sterno-clavicular joints, may accompany the skin and bone lesions. Skin involvement includes palmoplantar pustulosis, severe cystic acne and pustular psoriasis. Skin lesions may precede, occur concurrently or appear after onset of osteoarticular manifestations, but may be initially absent in 1/3 of patients. The interval bet

In [39]:
print(result['answer'])


SAPHO syndrome is a complex disorder characterized by peculiar combinations of bone lesions and dermatologic manifestations. Patients typically present with musculo-skeletal complaints, usually of the anterior chest wall, synovial articulations and synchondroses. The disease is more common in the sterno-clavicular region, but can also affect the spine, pelvis and long bones. Various articular manifestations, often due to osteitis extending to adjacent articular structures, especially in the sacroiliac and sterno-clavicular joints, may accompany the skin and bone lesions. Skin involvement includes palmoplantar pustulosis, severe cystic acne and pustular psoriasis. Skin lesions may precede, occur concurrently or appear after onset of osteoarticular manifestations, but may be initially absent in 1/3 of patients. The interval between onset of skin manifestations and osteoarticular lesions is generally less than 2 years, but intervals as long as 20 and 38 years have been reported. Hyperosto

In [28]:
print(result['sources'])

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2790148/, https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5038107/
